In [2]:
pip install lightgbm

In [3]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [4]:
cus = pd.read_csv("C:/Users/82104/Desktop/open/cus_info.csv")
iem = pd.read_csv("C:/Users/82104/Desktop/open/iem_info.csv")
hist = pd.read_csv("C:/Users/82104/Desktop/open/stk_bnc_hist.csv")
train = pd.read_csv("C:/Users/82104/Desktop/open/stk_hld_train.csv")
test = pd.read_csv("C:/Users/82104/Desktop/open/stk_hld_test.csv")

submission = pd.read_csv("C:/Users/82104/Desktop/open/sample_submission.csv")

In [5]:
train["hist_d"] = train["hold_d"]*0.6
train.hist_d = np.trunc(train["hist_d"])
# train 데이터에는 test 데이터에 있는 hist_d 컬럼이 없기 때문에 임의로 생성하는 과정.


In [6]:
train.head(3) # train 데이터에 임의로 생성한 hist_d 확인 ( 보유기간 * 0.6 )

,act_id,iem_cd,byn_dt,hold_d,hist_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0


In [7]:
train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test, iem, how = "left", on = ["iem_cd"])

# train 데이터와 test 데이터에 고객정보와 주식정보를 추가하는 과정.

In [8]:
train_data.head(3)

,act_id,iem_cd,byn_dt,hold_d,hist_d,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0,코스모신소재,12,2,99


In [9]:
train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

In [10]:
train_data.head(3)

,act_id,iem_cd,byn_dt,hist_d,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,3.0,코스모신소재,12,2,99


In [11]:

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [12]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [13]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 4.21439	valid_0's l2: 17.7611
[1000]	valid_0's rmse: 4.05912	valid_0's l2: 16.4765
[1500]	valid_0's rmse: 4.04072	valid_0's l2: 16.3274
[2000]	valid_0's rmse: 4.03876	valid_0's l2: 16.3116
Did not meet early stopping. Best iteration is:
[1718]	valid_0's rmse: 4.03614	valid_0's l2: 16.2904
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.32651	valid_0's l2: 1.75962
Early stopping, best iteration is:
[685]	valid_0's rmse: 1.26441	valid_0's l2: 1.59873
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.682567	valid_0's l2: 0.465898
[1000]	valid_0's rmse: 0.472995	valid_0's l2: 0.223725
Early stopping, best iteration is:
[823]	valid_0's rmse: 0.47138	valid_0's l2: 0.222199
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.573799	valid_0's l2: 0.329245
[1000]	valid_0's rmse: 0.414093	valid_0's l2: 0.17

In [14]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [15]:
predict

array([254.79927029, 557.53350053, 232.26625441, ..., 895.50353813,
        18.99762344,   7.46634223])

In [16]:
submission["hold_d"] = np.round(predict)

In [17]:
submission.to_csv("dacon_baseline.csv", index = False)